marked notebook as Trusted to display the map

#### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import pyodbc
import geopandas as gpd
import sqlalchemy as sal
from shapely import wkb
import folium
from folium import plugins
import contextily as ctx

the code for coloring different polygons random colors is adapted from
https://shallowsky.com/blog/mapping/ 

#### Connect to server

In [2]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};' 
                      'Server=NE-SQL-CL1-03.kf-ext.com;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

#### Get the data

In [3]:
SQLdata = """
SELECT [Submarket]
      ,[Market]
      ,[SpatialObj].STAsBinary() as Boundary
  FROM [GeoData].[dbo].[Commercial_LondonOffice_SubMarkets]       
"""

df = gpd.GeoDataFrame.from_postgis(SQLdata,conn, geom_col = 'Boundary', crs="EPSG:4326") 

In [4]:
df

,Submarket,Market,Boundary
0,Strand/Covent Garden,West End,"POLYGON ((-0.11801 51.50979, -0.11920 51.51132..."
1,West End Core,West End,"POLYGON ((-0.15228 51.50304, -0.15119 51.50328..."
2,Bloomsbury,West End,"POLYGON ((-0.12200 51.51737, -0.12151 51.51745..."
3,Marylebone,West End,"POLYGON ((-0.16581 51.52838, -0.16674 51.52920..."
4,Soho,West End,"POLYGON ((-0.13333 51.51010, -0.13215 51.51031..."
5,Fitzrovia,West End,"POLYGON ((-0.13039 51.51644, -0.13045 51.51659..."
6,Midtown,City,"POLYGON ((-0.11225 51.52204, -0.11197 51.52231..."
7,Euston/King's Cross,West End,"POLYGON ((-0.14852 51.52480, -0.14822 51.52418..."
8,Paddington,West End,"POLYGON ((-0.17008 51.51993, -0.17043 51.52012..."
9,Canary Wharf,Docklands,"POLYGON ((-0.00876 51.50124, -0.00872 51.50172..."


In [5]:
# Coordinate Reference System
df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
# very simple overview of search areas
#df.plot()

#### Get Folium map

In [7]:
m = folium.Map(location=(51.515, -0.10), zoom_start=13)

In [8]:
#m

#### Add polygons to Folium map

In [9]:
## custom function for random colors

import random

def random_html_color():
    r = random.randint(0,256)
    g = random.randint(0,256)
    b = random.randint(0,256)
    return '#%02x%02x%02x' % (r, g, b)

def style_fcn(x):
    return { 'fillColor': random_html_color(), 'fillOpacity': 0.8 }

In [10]:
#folium.GeoJson(df['Boundary'], style_function=style_fcn ).add_to(m)

#### Add pop-up names

code adapted from https://geopandas.readthedocs.io/en/latest/gallery/polygon_plotting_with_folium.html

In [11]:
for _, r in df.iterrows():
    boundary = folium.GeoJson(r['Boundary'], style_function=style_fcn )
    folium.Popup(r['Submarket']).add_to(boundary)
    boundary.add_to(m)
m

In [12]:
#style_fcn(1)

In [13]:
#m

In [14]:
#tooltip = folium.GeoJsonTooltip(fields=['Submarket'])